<a href="https://colab.research.google.com/github/jannikhoesch/fertilizers/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score
from joblib import dump
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import numpy as np

In [ ]:
# Read files
train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
train = train.drop(columns=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

## EDA

In [ ]:
print(train_df.shape)
print(train_df.dtypes)

(750000, 9)
Temparature         int64
Humidity            int64
Moisture            int64
Soil Type          object
Crop Type          object
Nitrogen            int64
Potassium           int64
Phosphorous         int64
Fertilizer Name    object
dtype: object


In [ ]:
train_df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,27,69,65,Sandy,Millets,30,6,18,28-28
2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,35,58,43,Red,Paddy,37,2,16,DAP


In [ ]:
train_df.describe()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,31.503565,61.038912,45.184147,23.093808,9.478296,21.073227
std,4.025574,6.647695,11.794594,11.216125,5.765622,12.346831
min,25.000000,50.000000,25.000000,4.000000,0.000000,0.000000
25%,28.000000,55.000000,35.000000,13.000000,4.000000,10.000000
50%,32.000000,61.000000,45.000000,23.000000,9.000000,21.000000
75%,35.000000,67.000000,55.000000,33.000000,14.000000,32.000000
max,38.000000,72.000000,65.000000,42.000000,19.000000,42.000000


## Preprocessing

In [ ]:
# Preprocess the data
X = train_df.drop(columns='Fertilizer Name')
y = train_df['Fertilizer Name']
X_test = test_df.copy()

In [ ]:
# Encode labels to integers
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [ ]:
# One-hot encode categorical variables
X_full = pd.concat([X, X_test], axis=0)
X_full_encoded = pd.get_dummies(X_full)
X_enc = X_full_encoded.iloc[:len(X)]
X_test_enc = X_full_encoded.iloc[len(X):]

In [ ]:
# Create folds
X_train, X_val, y_train, y_val = train_test_split(X_enc, y_enc, test_size=0.2, random_state=42)

## Train

In [ ]:
# Hyperparameters
params = {
    'learning_rate': 0.0404,
    'max_depth': 20,
    'min_child_weight': 5.53,
    'gamma': 0.2846,
    'alpha': 2.95,
    'subsample': 0.6,
    'colsample_bytree': 0.42,
    'eta': 0.5272,
    'n_estimators': 893,
    'lambda': 0.0106,
    'verbosity': 0,
    'objective': 'multi:softprob',
    'enable_categorical': False,
    'tree_method': "hist",
    'n_jobs': -1,
    'random_state': 42
}

In [ ]:
# Train model
model = XGBClassifier(**params)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)

[0]	validation_0-mlogloss:1.94542
[100]	validation_0-mlogloss:1.92042
[200]	validation_0-mlogloss:1.91150
[300]	validation_0-mlogloss:1.90738
[400]	validation_0-mlogloss:1.90608
[500]	validation_0-mlogloss:1.90619
[600]	validation_0-mlogloss:1.90714
[700]	validation_0-mlogloss:1.90886
[800]	validation_0-mlogloss:1.91065
[892]	validation_0-mlogloss:1.91246


XGBClassifier(alpha=2.95, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.42, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.5272, eval_metric=None,
              feature_types=None, gamma=0.2846, grow_policy=None,
              importance_type=None, interaction_constraints=None, lambda=0.0106,
              learning_rate=0.0404, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=20,
              max_leaves=None, min_child_weight=5.53, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=893, ...)

In [ ]:
# Validation predictions
val_labels = model.predict(X_val)
val_probs = model.predict_proba(X_val)

acc = accuracy_score(y_val, val_labels)
print(f"Accuracy: {acc:.4f}")

#val_top3_idx = val_probs.argsort(axis=1)[:, -3:][:, ::-1]
#val_top3_labels = le.inverse_transform(val_top3_idx.ravel()).reshape(val_top3_idx.shape)
#map3_score = mapk(y.values, val_top3_labels, k=3)
#print(f"MAP@3: {map3_score:.5f}")

Accuracy: 0.2098


In [ ]:
# Predict test set
test_probs = model.predict_proba(X_test_enc)
test_top3_idx = test_probs.argsort(axis=1)[:, -3:][:, ::-1]
test_top3_labels = le.inverse_transform(test_top3_idx.ravel()).reshape(test_top3_idx.shape)

In [ ]:
# Save test predictions
top3_str = [' '.join(le3.inverse_transform(row)) for row in top3_indices]
submission = pd.DataFrame({
    'id': test['id'],
    'Fertilizer Name': top3_str
})
submission.to_csv("submission.csv", index=False)
submission.head()

NameError: name 'pd' is not defined